In [1]:
import pandas as pd
import numpy as np

import datetime
from datetime import date, timedelta

import requests

import time
import math

import mysql.connector
import random

import seaborn as sns
import matplotlib.pyplot as plt

## Conectar con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Obtener cryptopunks de la BD

In [3]:
# Obtain cp and their types
sql = "SELECT `cryptopunks`.`id`, `cryptopunks`.`bids`, `cryptopunks`.`for_sale`, `ctypes`.`name`, `attributes`.`name`" \
        "FROM `cryptopunks` INNER JOIN `ctypes` ON `cryptopunks`.`id_type` = `ctypes`.`id`" \
        "LEFT JOIN `cryptopunk_has_attributes` ON `cryptopunks`.`id` = `cryptopunk_has_attributes`.`id_cryptopunk`" \
        "LEFT JOIN `attributes` ON `cryptopunk_has_attributes`.`id_attribute` = `attributes`.`id`"


mycursor.execute(sql)
por_tipo = mycursor.fetchall()

df_todos_cp = pd.DataFrame(por_tipo, columns = ['id_cryptopunk', 'bids', 'for_sale', 'type', 'attribute'])

df_todos_cp = df_todos_cp.sort_values('id_cryptopunk').reset_index(drop=True).copy()

print(df_todos_cp)

       id_cryptopunk  bids  for_sale    type         attribute
0                  0     1         0  Female        Blonde Bob
1                  0     1         0  Female           Earring
2                  0     1         0  Female  Green Eye Shadow
3                  1     0         0    Male             Smile
4                  1     0         0    Male            Mohawk
...              ...   ...       ...     ...               ...
27534           9998     0         1  Female   Wild White Hair
27535           9998     0         1  Female    Black Lipstick
27536           9998     0         1  Female  Clown Eyes Green
27537           9999     0         0  Female            Mohawk
27538           9999     0         0  Female      Nerd Glasses

[27539 rows x 5 columns]


### Obtener skin

In [4]:
f = open("ranks.csv", "r")

# creo el dataframe vacío
rank_df = pd.DataFrame(columns = ['id_cryptopunk', 'skin'])

# leo la primera línea
lin = f.readline()

In [5]:
while lin:
    vals = lin.split(',')
    partes = vals[2].split('>')
    cp = partes[1].split('<')[0]
    skin = vals[-3]
    
    # Guardo los datos en el df
    rank_df = rank_df.append({'id_cryptopunk': cp, 'skin': skin}, ignore_index=True)
    
    
    lin = f.readline()
f.close() 
rank_df = rank_df.astype({'id_cryptopunk': 'int32'})
rank_df = rank_df.sort_values('id_cryptopunk').reset_index(drop=True).copy()
rank_df

,id_cryptopunk,skin
0,0,Mid
1,1,Dark
2,2,Light
3,3,Dark
4,4,Mid
...,...,...
9995,9995,Albino
9996,9996,Light
9997,9997,Zombie
9998,9998,Mid


In [6]:
def obtener_skin(cp):
    val = rank_df[rank_df.id_cryptopunk == cp].skin
    return val.values[0]
    
    
obtener_skin(9998)

'Mid'

In [7]:
df_todos_cp['skin'] = df_todos_cp.apply(lambda row: obtener_skin(row.id_cryptopunk), axis=1)
df_todos_cp['new_type'] = df_todos_cp.type + '/' + df_todos_cp.skin

In [8]:
df_todos_cp.drop(['type', 'skin'], inplace=True, axis=1)
df_todos_cp.rename(columns={"new_type": "type"}, inplace=True)

## Obtener transacciones de la BD

In [9]:
# Obtener todas las transacciones
sql = "SELECT transactions.id, transactions.id_cryptopunk, cryptopunks.id_type, transactions.tr_type, transactions.amount, transactions.amount_dols, " \
        "transactions.txn FROM transactions,  cryptopunks WHERE " \
        "transactions.id_cryptopunk = cryptopunks.id "

mycursor.execute(sql)
transac = mycursor.fetchall()

df_transac = pd.DataFrame(transac, columns = ['tr_id', 'id_cryptopunk', 'tipo', 'tr_type', 'amount', 'amount_dols', 'date' ])

sql = "SELECT * FROM ctypes"

mycursor.execute(sql)
tipos = mycursor.fetchall()

tipos = dict(tipos)
tipos

df_transac['tipo'] = df_transac['tipo'].map(tipos, na_action=None)

df_transac['skin'] = df_transac.apply(lambda row: obtener_skin(row.id_cryptopunk), axis=1)
df_transac['new_type'] = df_transac.tipo + '/' + df_transac.skin
df_transac.drop(['tipo', 'skin'], inplace=True, axis=1)
df_transac.rename(columns={"new_type": "tipo"}, inplace=True)

df_transac = df_transac.sort_values(['id_cryptopunk','date']).reset_index(drop=True).copy()

df_transac

,tr_id,id_cryptopunk,tr_type,amount,amount_dols,date,tipo
0,84306,0,Sold,0.98,320.0,2017-06-23,Female/Mid
1,84307,0,Bid,0.98,320.0,2017-06-23,Female/Mid
2,84308,0,Bid,0.67,219.0,2017-06-23,Female/Mid
3,84309,0,Claimed,0.00,0.0,2017-06-23,Female/Mid
4,84305,0,Bid,0.05,14.0,2017-06-25,Female/Mid
...,...,...,...,...,...,...,...
184839,211535,9999,Bid Withdrawn,10.00,26529.0,2021-08-02,Female/Dark
184840,211534,9999,Bid,0.03,119.0,2021-12-17,Female/Dark
184841,211533,9999,Bid Withdrawn,0.03,116.0,2021-12-18,Female/Dark
184842,211531,9999,Bid Withdrawn,0.10,332.0,2022-01-15,Female/Dark


## Limpieza de transacciones
Elimino las que tienen monto 0 y las Wrap y Unwrap

In [10]:
df_mask = (df_transac['tr_type'] != '(Wrap)') & (df_transac['tr_type'] != '(UnWrap)') & (df_transac['amount'] >0)
df_transac = df_transac[df_mask].reset_index(drop=True).copy()
df_transac

,tr_id,id_cryptopunk,tr_type,amount,amount_dols,date,tipo
0,84306,0,Sold,0.98,320.0,2017-06-23,Female/Mid
1,84307,0,Bid,0.98,320.0,2017-06-23,Female/Mid
2,84308,0,Bid,0.67,219.0,2017-06-23,Female/Mid
3,84305,0,Bid,0.05,14.0,2017-06-25,Female/Mid
4,84303,0,Bid,1.30,369.0,2017-07-02,Female/Mid
...,...,...,...,...,...,...,...
140782,211535,9999,Bid Withdrawn,10.00,26529.0,2021-08-02,Female/Dark
140783,211534,9999,Bid,0.03,119.0,2021-12-17,Female/Dark
140784,211533,9999,Bid Withdrawn,0.03,116.0,2021-12-18,Female/Dark
140785,211531,9999,Bid Withdrawn,0.10,332.0,2022-01-15,Female/Dark


## Obtengo las últimas ofertas y últimas ventas
(aunque no esté en venta obtengo el último valor transaccionado)

In [11]:
def obtener_ultimas_tr(df, tipo):
    df_mask = df['tr_type']==tipo
    df = df[df_mask]

    df_ultimas = pd.DataFrame()
    ultimo_cp = min(df['id_cryptopunk'])
    max_fecha = min(df['date'])


    # ordeno el df de ofertas
    df = df.sort_values(by=['id_cryptopunk', 'date', 'tr_id'], ascending = [True, True, False]).copy()
    df = df.reset_index(drop=True).copy()

    linea_anterior = df.iloc[0]
    linea_ant_ant = linea_anterior

    for i in df.index:
        lin = df.iloc[i]
        if lin['id_cryptopunk'] != linea_anterior['id_cryptopunk']:
            if lin['amount'] != 0:
                df_ultimas = df_ultimas.append(linea_anterior)
            else:
                df_ultimas = df_ultimas.append(linea_ant_ant)
        linea_ant_ant = linea_anterior
        linea_anterior = lin
    df_ultimas = df_ultimas.append(linea_anterior)
    df_ultimas['id_cryptopunk'] =df_ultimas[['id_cryptopunk']].astype(int)
    return df_ultimas    
    
    
    


df_ult_ventas = obtener_ultimas_tr(df_transac,'Sold')
df_ult_ventas = df_ult_ventas.reset_index(drop=True).copy()
print(df_ult_ventas)

df_ult_ofertas = obtener_ultimas_tr(df_transac,'Offered').sort_values('id_cryptopunk').reset_index(drop=True).copy()
print(df_ult_ofertas)

          amount  amount_dols        date  id_cryptopunk           tipo  \
0      25.000000       2822.0  2018-11-30              0     Female/Mid   
1      60.000000      36305.0  2020-11-30              1      Male/Dark   
2       2.500000        234.0  2018-12-07             14    Female/Dark   
3       0.510000        105.0  2017-07-10             33     Male/Light   
4     129.990005     432676.0  2021-08-23             53   Female/Light   
...          ...          ...         ...            ...            ...   
6575  135.000000     403557.0  2021-09-30           9982   Female/Light   
6576    0.300000        140.0  2017-11-28           9989    Female/Dark   
6577    0.010000          0.0  2021-08-15           9990    Male/Albino   
6578   99.990000     169770.0  2021-02-08           9997  Zombie/Zombie   
6579  110.000000     346575.0  2021-08-11           9998     Female/Mid   

         tr_id tr_type  
0      84291.0    Sold  
1         12.0    Sold  
2        266.0    Sold  

## Poner el precio de venta en los cp que se venden

In [12]:
df_cp_con_precio = pd.merge(df_todos_cp, df_ult_ofertas, on='id_cryptopunk', how='outer')
df_cp_con_precio = df_cp_con_precio.drop(columns = ['tipo', 'tr_id', 'bids'])
df_cp_con_precio

,id_cryptopunk,for_sale,attribute,type,amount,amount_dols,date,tr_type
0,0,0,Blonde Bob,Female/Mid,100.0,1.830300e+04,2018-09-12,Offered
1,0,0,Earring,Female/Mid,100.0,1.830300e+04,2018-09-12,Offered
2,0,0,Green Eye Shadow,Female/Mid,100.0,1.830300e+04,2018-09-12,Offered
3,1,0,Smile,Male/Dark,79.5,4.684200e+04,2020-11-25,Offered
4,1,0,Mohawk,Male/Dark,79.5,4.684200e+04,2020-11-25,Offered
...,...,...,...,...,...,...,...,...
27534,9998,1,Wild White Hair,Female/Mid,250000.0,1.070000e+09,2021-10-29,Offered
27535,9998,1,Black Lipstick,Female/Mid,250000.0,1.070000e+09,2021-10-29,Offered
27536,9998,1,Clown Eyes Green,Female/Mid,250000.0,1.070000e+09,2021-10-29,Offered
27537,9999,0,Mohawk,Female/Dark,16.6,3.273000e+03,2017-07-14,Offered


In [13]:
# El cp 741 no tiene atributos
df_cp_con_precio[(df_cp_con_precio['id_cryptopunk'] ==741)]


,id_cryptopunk,for_sale,attribute,type,amount,amount_dols,date,tr_type
1696,741,1,None,Male/Dark,3000.0,12260000.0,2021-10-21,Offered


In [14]:
# El cp 2 no tiene ofertas
df_cp_con_precio[(df_cp_con_precio['id_cryptopunk'] ==2)]

,id_cryptopunk,for_sale,attribute,type,amount,amount_dols,date,tr_type
5,2,0,Wild Hair,Female/Light,NaN,NaN,NaN,NaN


## Convertir las fechas a rango y los tipos en dummies

In [15]:
# Obtener las fechas como rango para usar en regresión

df_ventas_rango = df_transac[(df_transac['tr_type']=='Sold')].reset_index().copy()
df_ventas_rango = df_ventas_rango.drop(['tr_type','tr_id'], axis=1)

df_ventas_rango = df_ventas_rango.rename(columns={"amount": "venta_E", "amount_dols": "venta_dols"})  
#df_ventas_rango['rango_fecha'] = datetime(df_ventas_rango['date']).dt.to_period('M')

df_ventas_rango['date'] = pd.to_datetime(df_ventas_rango['date'])
df_ventas_rango['rango_fecha'] =df_ventas_rango['date'].dt.to_period('M')

df_ventas_rango

,index,id_cryptopunk,venta_E,venta_dols,date,tipo,rango_fecha
0,0,0,0.98,320.0,2017-06-23,Female/Mid,2017-06
1,11,0,1.60,386.0,2017-07-07,Female/Mid,2017-07
2,15,0,25.00,2822.0,2018-11-30,Female/Mid,2018-11
3,43,1,0.42,107.0,2017-06-26,Male/Dark,2017-06
4,59,1,31.00,5155.0,2019-04-06,Male/Dark,2019-04
...,...,...,...,...,...,...,...
20044,140701,9997,59.00,21062.0,2020-09-27,Zombie/Zombie,2020-09
20045,140703,9997,99.99,169770.0,2021-02-08,Zombie/Zombie,2021-02
20046,140718,9998,15.00,9499.0,2020-12-27,Female/Mid,2020-12
20047,140722,9998,38.00,61170.0,2021-03-03,Female/Mid,2021-03


In [16]:
# Creo un diccionario para convertir el rango de fechas en un nro
años = range(2017,2023)
meses = range(1,13)
rango = list()

for a in años:
    for m in meses:
        sa =str(a)
        if len(str(m))==1:
            sm='0'+str(m)
        else:
            sm = str(m)
            
        rango.append(sa+'-'+ sm)
#print(rango)

nros = range(1, 12*6)

dicc = dict(zip(rango, nros))
#dicc

In [17]:

df_ventas_rango['rango_fecha'] = df_ventas_rango['rango_fecha'].astype('str')
df_ventas_rango['pos_fecha'] =df_ventas_rango['rango_fecha'].map(dicc)
df_ventas_rango = df_ventas_rango.drop(['rango_fecha', 'index'], axis=1)
df_ventas_rango


,id_cryptopunk,venta_E,venta_dols,date,tipo,pos_fecha
0,0,0.98,320.0,2017-06-23,Female/Mid,6
1,0,1.60,386.0,2017-07-07,Female/Mid,7
2,0,25.00,2822.0,2018-11-30,Female/Mid,23
3,1,0.42,107.0,2017-06-26,Male/Dark,6
4,1,31.00,5155.0,2019-04-06,Male/Dark,28
...,...,...,...,...,...,...
20044,9997,59.00,21062.0,2020-09-27,Zombie/Zombie,45
20045,9997,99.99,169770.0,2021-02-08,Zombie/Zombie,50
20046,9998,15.00,9499.0,2020-12-27,Female/Mid,48
20047,9998,38.00,61170.0,2021-03-03,Female/Mid,51


In [18]:
df_dumm = pd.get_dummies(df_ventas_rango['tipo'])
df_ventas_rango = pd.concat([df_ventas_rango, df_dumm], axis=1)
df_ventas_rango = df_ventas_rango.drop(['tipo'], axis =1)
df_ventas_rango

,id_cryptopunk,venta_E,venta_dols,date,pos_fecha,Alien/Alien,Ape/Ape,Female/Albino,Female/Dark,Female/Light,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie
0,0,0.98,320.0,2017-06-23,6,0,0,0,0,0,1,0,0,0,0,0
1,0,1.60,386.0,2017-07-07,7,0,0,0,0,0,1,0,0,0,0,0
2,0,25.00,2822.0,2018-11-30,23,0,0,0,0,0,1,0,0,0,0,0
3,1,0.42,107.0,2017-06-26,6,0,0,0,0,0,0,0,1,0,0,0
4,1,31.00,5155.0,2019-04-06,28,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,9997,59.00,21062.0,2020-09-27,45,0,0,0,0,0,0,0,0,0,0,1
20045,9997,99.99,169770.0,2021-02-08,50,0,0,0,0,0,0,0,0,0,0,1
20046,9998,15.00,9499.0,2020-12-27,48,0,0,0,0,0,1,0,0,0,0,0
20047,9998,38.00,61170.0,2021-03-03,51,0,0,0,0,0,1,0,0,0,0,0


## Armar ventas con atributos y tipos. 

In [19]:
df_ventas = df_transac[(df_transac['tr_type']=='Sold')]
df_ventas

,tr_id,id_cryptopunk,tr_type,amount,amount_dols,date,tipo
0,84306,0,Sold,0.98,320.0,2017-06-23,Female/Mid
11,84299,0,Sold,1.60,386.0,2017-07-07,Female/Mid
15,84291,0,Sold,25.00,2822.0,2018-11-30,Female/Mid
43,56,1,Sold,0.42,107.0,2017-06-26,Male/Dark
59,38,1,Sold,31.00,5155.0,2019-04-06,Male/Dark
...,...,...,...,...,...,...,...
140701,211480,9997,Sold,59.00,21062.0,2020-09-27,Zombie/Zombie
140703,211466,9997,Sold,99.99,169770.0,2021-02-08,Zombie/Zombie
140718,211528,9998,Sold,15.00,9499.0,2020-12-27,Female/Mid
140722,211524,9998,Sold,38.00,61170.0,2021-03-03,Female/Mid


In [20]:
# Obtain cp and their types
sql = "SELECT `cryptopunks`.`id`, `cryptopunks`.`bids`, `cryptopunks`.`for_sale`, `ctypes`.`name`" \
        "FROM `cryptopunks` INNER JOIN `ctypes` ON `cryptopunks`.`id_type` = `ctypes`.`id`" 


mycursor.execute(sql)
por_tp = mycursor.fetchall()

df_cp = pd.DataFrame(por_tp, columns = ['id_cryptopunk', 'bids', 'for_sale', 'type'])

df_cp = df_cp.sort_values('id_cryptopunk').reset_index(drop=True).copy()

print(df_cp)

      id_cryptopunk  bids  for_sale    type
0                 0     1         0  Female
1                 1     0         0    Male
2                 2     0         0  Female
3                 3     0         0    Male
4                 4     0         0    Male
...             ...   ...       ...     ...
9995           9995     0         0  Female
9996           9996     0         0    Male
9997           9997     0         0  Zombie
9998           9998     0         1  Female
9999           9999     0         0  Female

[10000 rows x 4 columns]


In [21]:
# Obtener una lista de todos los atributos

sql = "SELECT DISTINCT attributes.name FROM attributes INNER JOIN cryptopunk_has_attributes ON " \
            " attributes.id = cryptopunk_has_attributes.id_attribute"

mycursor.execute(sql)
atts = mycursor.fetchall()
atts

l = list()
for at in atts:
    l.append(at[0])


l = sorted(l)
l

['3D Glasses',
 'Bandana',
 'Beanie',
 'Big Beard',
 'Big Shades',
 'Black Lipstick',
 'Blonde Bob',
 'Blonde Short',
 'Blue Eye Shadow',
 'Buck Teeth',
 'Cap',
 'Cap Forward',
 'Chinstrap',
 'Choker',
 'Cigarette',
 'Classic Shades',
 'Clown Eyes Blue',
 'Clown Eyes Green',
 'Clown Hair Green',
 'Clown Nose',
 'Cowboy Hat',
 'Crazy Hair',
 'Dark Hair',
 'Do-rag',
 'Earring',
 'Eye Mask',
 'Eye Patch',
 'Fedora',
 'Front Beard',
 'Front Beard Dark',
 'Frown',
 'Frumpy Hair',
 'Goat',
 'Gold Chain',
 'Green Eye Shadow',
 'Half Shaved',
 'Handlebars',
 'Headband',
 'Hoodie',
 'Horned Rim Glasses',
 'Hot Lipstick',
 'Knitted Cap',
 'Luxurious Beard',
 'Medical Mask',
 'Messy Hair',
 'Mohawk',
 'Mohawk Dark',
 'Mohawk Thin',
 'Mole',
 'Mustache',
 'Muttonchops',
 'Nerd Glasses',
 'Normal Beard',
 'Normal Beard Black',
 'Orange Side',
 'Peak Spike',
 'Pigtails',
 'Pilot Helmet',
 'Pink With Hat',
 'Pipe',
 'Police Cap',
 'Purple Eye Shadow',
 'Purple Hair',
 'Purple Lipstick',
 'Red Mohawk'

In [22]:
filas = range(0,10000)
df_attr = pd.DataFrame(columns=l, index=filas)
df_attr=df_attr.fillna(0)

sql = "SELECT cryptopunk_has_attributes.id_cryptopunk, attributes.name FROM attributes INNER JOIN " \
            " cryptopunk_has_attributes ON attributes.id = cryptopunk_has_attributes.id_attribute"

mycursor.execute(sql)
cp_at = mycursor.fetchall()

for x in range(0,10000):
    df_attr.loc[x,'id_cryptopunk'] =x

for cp, at in cp_at:
    df_attr.loc[cp,at] =1
    
    

df_attr


,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,Buck Teeth,...,Tiara,Top Hat,VR,Vampire Hair,Vape,Welding Goggles,Wild Blonde,Wild Hair,Wild White Hair,id_cryptopunk
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,3.0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9995.0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9996.0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9997.0
9998,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9998.0


In [23]:
# Si quiero todas las ventas de cada cp
#df_cp_ventas = pd.merge(df_attr, df_ventas, on='id_cryptopunk', how='outer')

# Si quiero sólo la última venta de cada cp
df_cp_ventas = pd.merge(df_attr, df_ult_ventas, on='id_cryptopunk', how='outer')

df_cp_ventas = df_cp_ventas.drop(columns = ['tr_id', 'tr_type'])
df_cp_ventas

,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,Buck Teeth,...,Vape,Welding Goggles,Wild Blonde,Wild Hair,Wild White Hair,id_cryptopunk,amount,amount_dols,date,tipo
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0.0,25.00,2822.0,2018-11-30,Female/Mid
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.0,60.00,36305.0,2020-11-30,Male/Dark
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2.0,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,3.0,NaN,NaN,NaN,NaN
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9995.0,NaN,NaN,NaN,NaN
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9996.0,NaN,NaN,NaN,NaN
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9997.0,99.99,169770.0,2021-02-08,Zombie/Zombie
9998,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,9998.0,110.00,346575.0,2021-08-11,Female/Mid


In [24]:
df_dumm = pd.get_dummies(df_cp_ventas['tipo'])
df_ventas_attr = pd.concat([df_cp_ventas, df_dumm], axis=1)
df_ventas_attr = df_ventas_attr.drop(['tipo'], axis =1)

df_primera = pd.DataFrame(columns =['fecha'])
df_primera['fecha'] = df_ventas_attr.loc[:,'date']

df_ultima =  pd.DataFrame(columns = ['valor_$'])
df_ultima['valor_$'] = df_ventas_attr.loc[:,'amount_dols']

df_ventas_attr = pd.concat([df_primera, df_ventas_attr, df_ultima], axis=1)
df_ventas_attr =df_ventas_attr.drop(['date', 'amount', 'amount_dols'], axis=1)
df_ventas_attr


,fecha,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,...,Female/Albino,Female/Dark,Female/Light,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie,valor_$
0,2018-11-30,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,2822.0
1,2020-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,36305.0
2,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
9996,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
9997,2021-02-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,169770.0
9998,2021-08-11,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,346575.0


In [25]:
# Función para contar la cantidad de atributos de un cp
def cuantos_attr(cp):
    sql = "SELECT COUNT(id_attribute) FROM cryptopunk_has_attributes WHERE id_cryptopunk = %s"
    mycursor.execute(sql, (cp,))
    return mycursor.fetchall()[0][0]

df_ventas_attr['cant_attr'] = df_ventas_attr.apply(lambda row: cuantos_attr(row.id_cryptopunk), axis=1)    
df_ventas_attr



,fecha,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,...,Female/Dark,Female/Light,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie,valor_$,cant_attr
0,2018-11-30,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,2822.0,3
1,2020-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,36305.0,2
2,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,1
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,3
4,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,2
9996,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,4
9997,2021-02-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,169770.0,2
9998,2021-08-11,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,346575.0,3


In [26]:
# Función para obtener el beneficio de la última venta
def obtener_beneficio(cp):
    sql = "SELECT amount_dols FROM transactions WHERE id_cryptopunk = %s AND tr_type='Sold' ORDER BY txn DESC"
    mycursor.execute(sql, (cp,))
    ventas = mycursor.fetchall()
    cant_ventas = len(ventas)
    if cant_ventas<2: 
        benef = 0
    else:
        suma = 0
        for i in range(0,len(ventas)-1):
            suma = suma + (ventas[i][0] - ventas[i+1][0])
        benef = suma
    return benef


df_ventas_attr['beneficio'] = df_ventas_attr.apply(lambda row: obtener_beneficio(row.id_cryptopunk), axis=1)    
df_ventas_attr


,fecha,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,...,Female/Light,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie,valor_$,cant_attr,beneficio
0,2018-11-30,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,2822.0,3,2502.0
1,2020-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,36305.0,2,36198.0
2,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,1,0.0
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,3,0.0
4,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,NaN,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,2,0.0
9996,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,4,0.0
9997,2021-02-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,169770.0,2,148708.0
9998,2021-08-11,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,346575.0,3,337076.0


In [27]:
# Función para obtener la cantidad de ventas
def obtener_cant_ventas(cp):
    sql = "SELECT amount_dols FROM transactions WHERE id_cryptopunk = %s AND tr_type='Sold' ORDER BY txn DESC"
    mycursor.execute(sql, (cp,))
    ventas = mycursor.fetchall()
    cant_ventas = len(ventas)
    if cant_ventas<2: 
        benef = 0
    else:
        suma = 0
        for i in range(0,len(ventas)-1):
            suma = suma + (ventas[i][0] - ventas[i+1][0])
        benef = suma
    return cant_ventas


df_ventas_attr['cant_ventas'] = df_ventas_attr.apply(lambda row: obtener_cant_ventas(row.id_cryptopunk), axis=1)    
df_ventas_attr

,fecha,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,...,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie,valor_$,cant_attr,beneficio,cant_ventas
0,2018-11-30,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,2822.0,3,2502.0,3
1,2020-11-30,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,36305.0,2,36198.0,3
2,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,1,0.0,0
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,3,0.0,0
4,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,NaN,4,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,2,0.0,0
9996,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,4,0.0,0
9997,2021-02-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,169770.0,2,148708.0,2
9998,2021-08-11,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,346575.0,3,337076.0,3


In [28]:
tp = 'Female'
df_ventas_attr[(df_ventas_attr['id_cryptopunk'] == 9999)]

,fecha,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,Blonde Bob,Blonde Short,Blue Eye Shadow,...,Female/Mid,Male/Albino,Male/Dark,Male/Light,Male/Mid,Zombie/Zombie,valor_$,cant_attr,beneficio,cant_ventas
9999,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,2,0.0,0


## Bajar los df a csv

In [29]:
#df_todos_cp.to_csv('cryptopunks.csv')
df_transac.drop(columns = ['tr_id']).to_csv('transacciones.csv')
#df_ult_ofertas.to_csv('ultimas_oferta.csv')
#df_ult_ventas.to_csv('ultimas_ventas.csv')
df_cp_con_precio.to_csv('cryptopunks_con_precio_de_oferta.csv')
df_ventas_rango.to_csv('ventas_por_tipo_y_fecha.csv')
df_ventas_attr.to_csv('ventas_por_tipo_atributo_y_fecha.csv')